In [10]:
import os
import torch
from torch import optim, nn, utils, Tensor
import pandas as pd
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset
import numpy as np
torch.manual_seed(0)

In [11]:
from numpy import double
device = "cuda" if torch.cuda.is_available() else "cpu"

class SimpleNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(2, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, 5)
        self.fc5 = nn.Linear(5, 2)
        self.soft = nn.Softmax(dim = 0)
    
    def forward(self, x):
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        # x = self.soft(x)
        lossFunc = nn.CrossEntropyLoss()
        print(y)
        if y == torch.tensor(double(int("1"))):
            y = torch.tensor([0,1]).float()
        else:
            y = torch.tensor([1,0]).float()
        print(logits, y)
        loss = lossFunc(logits, y)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer


In [12]:
simpleNN = SimpleNN().to(device)
print(simpleNN)

SimpleNN(
  (fc1): Linear(in_features=2, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=5, bias=True)
  (fc5): Linear(in_features=5, out_features=2, bias=True)
  (soft): Softmax(dim=0)
)


In [13]:
X = list()
Y = list()
X.append(torch.Tensor([1,5]).float())
Y.append(torch.Tensor([0]).float())

X.append(torch.Tensor([2,3]).float())
Y.append(torch.Tensor([0]).float())

X.append(torch.Tensor([4,2]).float())
Y.append(torch.Tensor([1]).float())

X.append(torch.Tensor([7,3]).float())
Y.append(torch.Tensor([1]).float())

X.append(torch.Tensor([9,8]).float())
Y.append(torch.Tensor([1]).float())

X.append(torch.Tensor([8,9]).float())
Y.append(torch.Tensor([0]).float())

X.append(torch.Tensor([0,1]).float())
Y.append(torch.Tensor([0]).float())

X.append(torch.Tensor([2,0]).float())
Y.append(torch.Tensor([1]).float())

In [14]:

class stockDataset(Dataset):
    def __init__(self, X, Y):
        super().__init__()
        self.labels = Y
        self.data = X
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        print(self.labels[index])
        return self.data[index], self.labels[index]

In [15]:
dataset = stockDataset(X,Y)

In [16]:
train, test = torch.utils.data.random_split(dataset, [8,0])

In [17]:
train_loader = torch.utils.data.DataLoader(train, 1, shuffle = True)
# test_loader = torch.utils.data.DataLoader(test, 1, shuffle = True)

In [18]:
trainer = pl.Trainer(limit_train_batches=100, max_epochs=50)
trainer.fit(model=simpleNN, train_dataloaders=train_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/exitialium/miniconda3/envs/simpleNNstock/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name | Type    | Params
---------------------------------
0 | fc1  | Linear  | 150   
1 | fc2  | Linear  | 510   
2 | fc3  | Linear  | 220   
3 | fc4  | Linear  | 105   
4 | fc5  | Linear  | 12    
5 | soft | Softmax | 0     
---------------------------------
997       Trainable params
0         Non-trainable params
997       Total params
0.004     Total estimated model params size (MB)
/home/exitialium/miniconda3/envs/simpleNNstock/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, train_dataloader

Epoch 0:   0%|          | 0/8 [00:00<?, ?it/s] tensor([1.])
tensor([[1.]])
tensor([0.0472, 0.1216], grad_fn=<AddBackward0>) tensor([0., 1.])
Epoch 0:  12%|█▎        | 1/8 [00:00<00:00, 40.16it/s, loss=0.657, v_num=18]tensor([0.])
tensor([[0.]])
tensor([0.0367, 0.1173], grad_fn=<AddBackward0>) tensor([1., 0.])
Epoch 0:  25%|██▌       | 2/8 [00:00<00:00, 62.50it/s, loss=0.695, v_num=18]tensor([1.])
tensor([[1.]])
tensor([0.0461, 0.1236], grad_fn=<AddBackward0>) tensor([0., 1.])
Epoch 0:  38%|███▊      | 3/8 [00:00<00:00, 77.68it/s, loss=0.682, v_num=18]tensor([1.])
tensor([[1.]])
tensor([0.0150, 0.1383], grad_fn=<AddBackward0>) tensor([0., 1.])
Epoch 0:  50%|█████     | 4/8 [00:00<00:00, 85.30it/s, loss=0.67, v_num=18] tensor([1.])
tensor([[1.]])
tensor([-0.1288,  0.2085], grad_fn=<AddBackward0>) tensor([0., 1.])
Epoch 0:  62%|██████▎   | 5/8 [00:00<00:00, 90.13it/s, loss=0.644, v_num=18]tensor([0.])
tensor([[0.]])
tensor([-0.1478,  0.2157], grad_fn=<AddBackward0>) tensor([1., 0.])
Epoch

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 8/8 [00:00<00:00, 95.59it/s, loss=0.271, v_num=18] 


In [19]:
# load checkpoint
checkpoint = "./lightning_logs/version_18/checkpoints/epoch=49-step=400.ckpt"
simpleNN = SimpleNN.load_from_checkpoint(checkpoint).double()

# choose your trained nn.Module
simpleNN.eval().float()

SimpleNN(
  (fc1): Linear(in_features=2, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=5, bias=True)
  (fc5): Linear(in_features=5, out_features=2, bias=True)
  (soft): Softmax(dim=0)
)

In [20]:
sf = nn.Softmax(dim = 0)

In [12]:
print(Y[0])

0


In [21]:
for i in range(len(X)):
    print(sf(simpleNN(X[i])), Y[i])

tensor([0.5566, 0.4434], grad_fn=<SoftmaxBackward0>) tensor([0.])
tensor([0.5566, 0.4434], grad_fn=<SoftmaxBackward0>) tensor([0.])
tensor([0.0032, 0.9968], grad_fn=<SoftmaxBackward0>) tensor([1.])
tensor([7.5380e-05, 9.9992e-01], grad_fn=<SoftmaxBackward0>) tensor([1.])
tensor([0.0128, 0.9872], grad_fn=<SoftmaxBackward0>) tensor([1.])
tensor([0.5566, 0.4434], grad_fn=<SoftmaxBackward0>) tensor([0.])
tensor([0.5566, 0.4434], grad_fn=<SoftmaxBackward0>) tensor([0.])
tensor([0.0050, 0.9950], grad_fn=<SoftmaxBackward0>) tensor([1.])


In [31]:
x = torch.tensor([0,2]).float()
print(sf(simpleNN(x)))

tensor([0.5566, 0.4434], grad_fn=<SoftmaxBackward0>)


In [337]:
lossFunc = nn.CrossEntropyLoss()
input = torch.tensor([0.54,0.78])
target = torch.tensor([0,1]).float()
print(input)
print(target)
output = lossFunc(input,target)
print(output)

tensor([0.5400, 0.7800])
tensor([0., 1.])
tensor(0.5803)


In [401]:
simpleNN(torch.randn([104, 22], dtype=float))

tensor([-0.1216,  0.1657], dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np

# class Model(nn.Module):
#     """
#     Just one Linear layer
#     """
#     def __init__(self, configs):
#         super(Model, self).__init__()
#         self.seq_len = configs.seq_len
#         self.pred_len = configs.pred_len
#         self.Linear = nn.Linear(self.seq_len, self.pred_len)
#         # Use this line if you want to visualize the weights
#         # self.Linear.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

#     def forward(self, x):
#         # x: [Batch, Input length, Channel]
#         x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
#         return x # [Batch, Output length, Channel]